In [ ]:
!pip install gym
import numpy as np
import gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def run_game(env,policy,episodes):
  games_won=0
  for _ in range(episodes):
    state=env.reset()
    while True:
      action=policy[state]
      new_state,rew,done,_=env.step(action)
      games_won+=rew
      if done:
        break
  print("games won {}".format(games_won))
  return games_won

def train_agent_MC(episodes,env,Q,N,policy):
  epsilon=1
  state=env.reset()
  for i in range(episodes):
    total_rew=0
    episode_states=[]
    epsilon=max(0,epsilon-i/episodes)
    state=env.reset()
    done=False
    first_visit=[]
    while not done:
      if np.random.random()<1-epsilon:
        action=env.action_space.sample()
      else:
        action=np.argmax(Q[state])#off policy
      episode_states.append((state,action))
      state,rew,done,_=env.step(action)
      total_rew+=rew
    for s,a in episode_states:
      if (s,a) not in first_visit:
        N[s][a]+=1
        Q[s][a]+=(1/N[s][a])*(total_rew-Q[s][a])
        first_visit.append((s,a))
        
  for state in range(env.observation_space.n):
    policy[state]=np.argmax(Q[state])
  return policy


In [ ]:
if __name__=='__main__':
  episodes=100000
  total_game=100
  env=gym.make('FrozenLake-v0')
  Q=np.zeros([env.observation_space.n,env.action_space.n])
  N=np.zeros([env.observation_space.n,env.action_space.n])
  policy=np.zeros([env.observation_space.n])
  optimal_policy=train_agent_MC(episodes,env,Q,N,policy)
  print(optimal_policy)
  for _ in range(10):
    run_game(env,optimal_policy,total_game)

[1. 3. 0. 3. 0. 0. 0. 0. 3. 2. 0. 0. 0. 2. 1. 0.]
games won 5.0
games won 8.0
games won 4.0
games won 3.0
games won 8.0
games won 2.0
games won 2.0
games won 5.0
games won 2.0
games won 4.0
